In [ ]:
import pandas
import sklearn
import matplotlib.pyplot as plt
import xgboost
import pandas as pd

In [ ]:
train_df = pd.read_csv('../data/processed/train.csv')
valid_df = pd.read_csv('../data/processed/valid.csv')
test_df = pd.read_csv('../data/processed/test.csv')
train_df
print(pd.isnull(train_df).any())
print(pd.isnull(valid_df).any())
print(pd.isnull(test_df).any())

In [ ]:
X_train = train_df['clean_text']
y_train = train_df['sentiment']

X_val = valid_df['clean_text']
y_val = valid_df['sentiment']

X_test = test_df['clean_text']
y_test = test_df['sentiment']

### TD-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,3),max_features=10000, min_df=2,max_df=0.95)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf   = tfidf.transform(X_val)
X_test_tfidf  = tfidf.transform(X_test)

In [ ]:
print(X_train_tfidf)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = model.predict(X_val_tfidf)
score = accuracy_score(y_val, y_pred)
print(score)

cm = confusion_matrix(y_val,y_pred)
ConfusionMatrixDisplay.from_predictions(y_val, y_pred)
plt.show()

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

bst = XGBClassifier(booster = 'gbtree',n_estimators=2, max_depth=4, learning_rate=1, objective='binary:logistic', eval_metric='logloss')
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_val_le = le.transform(y_val)
y_test_le = le.transform(y_test)

bst.fit(X_train_tfidf,y_train_le)

In [ ]:
y_pred_xgb = bst.predict(X_val_tfidf)

In [ ]:
y_val_le

In [ ]:
score = accuracy_score(y_val_le, y_pred_xgb)
print(score)

cm = confusion_matrix(y_val_le, y_pred_xgb)
ConfusionMatrixDisplay.from_predictions(y_val_le, y_pred_xgb)
plt.show()